In [2]:
import rasterio as rio
import rioxarray as rxr
import geopandas as gpd
import shapely as geometry
from rasterio.crs import CRS
import os, shutil

In [3]:
def raster_grid_split(source):
    path = "shapefile_boundaries"
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)

    def _get_whole_boundaries(maxX, minX, maxY, minY):
        left_bottom = (minX, minY)
        left_top = (minX, maxY)
        right_top = (maxX, maxY)
        right_bottom = (maxX, minY)

        geom = geometry.Polygon([left_bottom, left_top, right_top, right_bottom])
        geomdf = gpd.GeoDataFrame(index=[0], geometry=[geom], crs="EPSG:4326")

        path_full_boundaries = os.path.join(path, "temp_raster_full_boundaries.geojson")
        geomdf.to_file(path_full_boundaries, driver="GeoJSON")

        return geomdf, path_full_boundaries
    
    imagery = rio.open(source)
    raster = rxr.open_rasterio(source, masked=True).squeeze()
    raster_new = raster.rio.reproject(CRS.from_string('EPSG:4326'))

    maxX = raster_new.coords['x'].max(dim=["x"]).to_dict()['data']
    minX = raster_new.coords['x'].min(dim=["x"]).to_dict()['data']
    maxY = raster_new.coords['y'].max(dim=["y"]).to_dict()['data']
    minY = raster_new.coords['y'].min(dim=["y"]).to_dict()['data']

    # Select grids within area study
    _, wbounds_path = _get_whole_boundaries(maxX, minX, maxY, minY)

    return wbounds_path

In [7]:
area = {
    "name": "tepilora",
    "path": "base_raster/tepilora/T32TNL_20160419T101032_B01.jp2"
}
res_path = raster_grid_split(source=area['path'])
shutil.move(res_path, os.path.join("shapefile_boundaries", f"{area['name']}_{os.path.basename(res_path)}"))

'shapefile_boundaries/tepilora_temp_raster_full_boundaries.geojson'